# Lab-1.7: DPO (Direct Preference Optimization) 環境與數據準備

**實驗目標**: 準備 DPO 訓練環境與偏好數據集

**Direct Preference Optimization (DPO)** 是一種新穎的 RLHF 替代方案，由 Stanford 提出，能夠直接從偏好數據中優化語言模型，無需訓練獎勵模型或執行強化學習。

## 核心概念

DPO 的主要優勢：
- **無需獎勵模型**: 直接從偏好數據訓練，簡化 RLHF 流程
- **穩定訓練**: 避免強化學習中的不穩定性問題
- **高效實現**: 結合 PEFT 技術，在單 GPU 上實現大模型對齊
- **理論保證**: 基於 Bradley-Terry 模型的理論基礎

## 實驗流程

1. **環境準備**: 安裝 TRL、PEFT 等必要庫
2. **數據準備**: 載入偏好數據集並進行預處理
3. **模型載入**: 準備基礎模型和 tokenizer
4. **數據格式化**: 將偏好對轉換為 DPO 訓練格式

---

## 步驟 1: 環境檢查與安裝

首先檢查 GPU 環境並安裝必要的套件。DPO 訓練需要：
- `trl`: Transformer Reinforcement Learning 庫
- `peft`: 參數高效微調
- `bitsandbytes`: 量化支持
- `accelerate`: 分散式訓練支持

In [ ]:
# 檢查 GPU 環境
import torch
import sys
import os

print(f'Python 版本: {sys.version}')
print(f'PyTorch 版本: {torch.__version__}')
print(f'CUDA 可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU 設備: {torch.cuda.get_device_name()}')
    print(f'GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
    print(f'當前記憶體使用: {torch.cuda.memory_allocated() / 1e9:.2f} GB')

# 設置隨機種子確保實驗可重現
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [ ]:
# 安裝必要套件 (如果尚未安裝)
import subprocess
import importlib

def install_and_import(package, import_name=None):
    if import_name is None:
        import_name = package
    
    try:
        importlib.import_module(import_name)
        print(f'✅ {package} 已安裝')
    except ImportError:
        print(f'⚠️  安裝 {package}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
        print(f'✅ {package} 安裝完成')

# 檢查必要套件
install_and_import('trl')
install_and_import('peft')
install_and_import('bitsandbytes')
install_and_import('accelerate')
install_and_import('transformers')
install_and_import('datasets')

In [ ]:
# 導入必要庫並檢查版本
import transformers
import peft
import trl
import datasets
import bitsandbytes
import accelerate

print('=== 套件版本資訊 ===')
print(f'🤗 Transformers: {transformers.__version__}')
print(f'🎯 PEFT: {peft.__version__}')
print(f'🚀 TRL: {trl.__version__}')
print(f'📊 Datasets: {datasets.__version__}')
print(f'⚡ Accelerate: {accelerate.__version__}')

print('✅ DPO 環境準備完成！')

## 步驟 2: 偏好數據集載入與探索

DPO 需要偏好數據，格式為 (prompt, chosen, rejected) 三元組：
- **prompt**: 輸入提示
- **chosen**: 偏好的回應 (高品質)
- **rejected**: 非偏好的回應 (低品質)

我們使用 Anthropic 的 HH-RLHF 數據集，這是人類偏好數據的經典數據集。

In [ ]:
from datasets import load_dataset
import pandas as pd

# 載入偏好數據集
print('📥 載入 Anthropic HH-RLHF 數據集...')

try:
    # 先載入小樣本進行測試
    dataset = load_dataset('Anthropic/hh-rlhf', split='train[:100]')
    print(f'✅ 成功載入數據集，樣本數: {len(dataset)}')
    
    # 檢查數據格式
    print('\n=== 數據集結構 ===')
    print(f'欄位: {list(dataset.features.keys())}')
    
except Exception as e:
    print(f'❌ 數據集載入失敗: {e}')
    print('使用本地模擬數據...')
    
    # 創建模擬偏好數據
    mock_data = {
        'chosen': [
            'Human: 請解釋什麼是機器學習?\n\nAssistant: 機器學習是人工智能的一個分支，它讓計算機能夠從數據中學習並做出決策，而無需明確編程。通過算法分析大量數據，系統可以識別模式並提高性能。',
            'Human: 如何學習程式設計?\n\nAssistant: 學習程式設計建議從基礎語法開始，選擇一門適合的語言如Python，多做練習項目，參與開源專案，並持續學習新技術。實作是最重要的學習方式。'
        ],
        'rejected': [
            'Human: 請解釋什麼是機器學習?\n\nAssistant: 機器學習就是讓機器變聰明。',
            'Human: 如何學習程式設計?\n\nAssistant: 學程式就是寫code。'
        ]
    }
    
    from datasets import Dataset
    dataset = Dataset.from_dict(mock_data)
    print(f'✅ 創建模擬數據集，樣本數: {len(dataset)}')

In [ ]:
# 探索數據集內容
print('=== 數據樣本示例 ===')

for i in range(min(2, len(dataset))):
    sample = dataset[i]
    print(f'\n📝 樣本 {i+1}:')
    
    if 'chosen' in sample:
        print('✅ 偏好回應 (chosen):')
        print(sample['chosen'][:200] + '...' if len(sample['chosen']) > 200 else sample['chosen'])
        print()
        
    if 'rejected' in sample:
        print('❌ 非偏好回應 (rejected):')
        print(sample['rejected'][:200] + '...' if len(sample['rejected']) > 200 else sample['rejected'])
        print()
    
    print('-' * 80)

## 步驟 3: 模型與 Tokenizer 準備

DPO 訓練需要一個預訓練的基礎模型。我們使用較小的模型進行實驗，並配置量化以節省記憶體。

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 選擇模型 (使用較小的模型進行實驗)
MODEL_NAME = 'microsoft/DialoGPT-medium'  # 或者使用 'gpt2' 進行快速測試

print(f'📦 載入模型: {MODEL_NAME}')

# 配置量化 (可選，用於大模型)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True
)

try:
    # 載入 tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # 設置 pad token (DPO 需要)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
    
    print(f'✅ Tokenizer 載入成功')
    print(f'詞彙大小: {len(tokenizer)}')
    print(f'特殊 tokens: EOS={tokenizer.eos_token}, PAD={tokenizer.pad_token}')
    
    # 載入模型 (不量化進行快速測試)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16,
        device_map='auto' if torch.cuda.is_available() else 'cpu'
        # quantization_config=quantization_config  # 如需量化請取消註解
    )
    
    print(f'✅ 模型載入成功')
    print(f'模型參數量: {model.num_parameters():,}')
    print(f'模型設備: {next(model.parameters()).device}')
    
except Exception as e:
    print(f'❌ 模型載入失敗: {e}')
    print('請檢查網路連接或嘗試其他模型')

## 步驟 4: 數據預處理與格式化

將偏好數據轉換為 DPO 訓練所需的格式。DPO 需要 prompt、chosen、rejected 三個欄位。

In [ ]:
def extract_prompt_and_response(text):
    """從對話文本中提取提示和回應"""
    if 'Human:' in text and 'Assistant:' in text:
        parts = text.split('Assistant:')
        if len(parts) >= 2:
            prompt = parts[0].replace('Human:', '').strip()
            response = parts[1].strip()
            return prompt, response
    return None, None

def preprocess_dataset(dataset):
    """預處理數據集為 DPO 格式"""
    processed_data = []
    
    for sample in dataset:
        # 提取 chosen 和 rejected 的 prompt 和 response
        chosen_prompt, chosen_response = extract_prompt_and_response(sample['chosen'])
        rejected_prompt, rejected_response = extract_prompt_and_response(sample['rejected'])
        
        if chosen_prompt and chosen_response and rejected_response:
            processed_data.append({
                'prompt': chosen_prompt,
                'chosen': chosen_response,
                'rejected': rejected_response
            })
    
    return processed_data

# 預處理數據
print('🔄 預處理偏好數據...')
processed_data = preprocess_dataset(dataset)

print(f'✅ 處理完成，有效樣本數: {len(processed_data)}')

# 顯示處理後的樣本
if processed_data:
    print('\n=== 處理後樣本示例 ===')
    sample = processed_data[0]
    print(f'📝 Prompt: {sample["prompt"]}')
    print(f'✅ Chosen: {sample["chosen"]}')
    print(f'❌ Rejected: {sample["rejected"]}')

In [ ]:
# 創建最終的 DPO 數據集
from datasets import Dataset

if processed_data:
    dpo_dataset = Dataset.from_list(processed_data)
    print(f'📊 DPO 數據集創建完成')
    print(f'樣本數: {len(dpo_dataset)}')
    print(f'欄位: {list(dpo_dataset.features.keys())}')
    
    # 保存預處理數據 (可選)
    output_dir = './dpo_data'
    os.makedirs(output_dir, exist_ok=True)
    
    dpo_dataset.save_to_disk(output_dir)
    print(f'💾 數據集已保存至: {output_dir}')
    
else:
    print('❌ 無有效的處理數據，請檢查數據格式')

## 步驟 5: 實驗設置檢查

最後確認所有組件準備就緒，為下一個 notebook 的 SFT 基線訓練做準備。

In [ ]:
# 實驗設置總結
print('=== DPO 實驗環境設置總結 ===')
print(f'✅ PyTorch 版本: {torch.__version__}')
print(f'✅ GPU 可用: {torch.cuda.is_available()}')
print(f'✅ 模型: {MODEL_NAME}')
print(f'✅ 數據集樣本數: {len(processed_data) if processed_data else 0}')
print(f'✅ Tokenizer 詞彙大小: {len(tokenizer)}')

# 測試 tokenization
test_text = 'Hello, how are you?'
tokens = tokenizer(test_text, return_tensors='pt')
print(f'✅ Tokenization 測試成功: {test_text} -> {tokens["input_ids"].shape[1]} tokens')

print('🚀 準備進入下一步: SFT 基線訓練')
print('請執行 02-SFT_Baseline.ipynb')